In [5]:
import pandas as pd

# Assuming your CSV file has columns 'id' and 'label'
csv_file_path = 'train.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Count the number of images for each label
label_counts = df['label'].value_counts()

# Count the number of unique labels
unique_labels_count = len(label_counts)

# Calculate the average number of images per label
average_images_per_label = label_counts.mean()

# Display the results
print(f"Number of unique labels: {unique_labels_count}")
print("\nNumber of images for each label:")
print(label_counts)
print(f"\nAverage number of images per label: {average_images_per_label:.2f}")


Number of unique labels: 100

Number of images for each label:
68    49
45    47
32    46
84    46
9     46
      ..
89    34
15    34
34    33
53    30
88    28
Name: label, Length: 100, dtype: int64

Average number of images per label: 41.35
